In [4]:
# 导入库
from IPython.display import Markdown as md
import sqlite3
import time
import pandas as pd
import numpy as np
import json
from copy import deepcopy

In [11]:
from IPython.display import Markdown as md
from matplotlib import pyplot as plt
import sqlite3
import time
import pandas as pd
import json



# 读取泰坦尼克号数据集
df = pd.read_csv('train.csv')
# 将表格中 NaN 值替换为 NULL
df = df.where(pd.notnull(df), None)

# 制作两个新的列：passengerInfo 和 ticketInfo
# 前者记录乘客的基本信息，后者记录乘客所持船票的信息
# 
passengerInfo_cols = ['Name','Sex','Age','SibSp','Parch']
ticketInfo_cols = ['Pclass','Ticket','Fare','Cabin','Embarked']

# 两个列分别以一个 json 字符串的形式储存对应的数据
#
df['passengerInfo'] = df[passengerInfo_cols].apply(lambda x: json.dumps(x.to_dict()),axis=1)
df['ticketInfo'] = df[ticketInfo_cols].apply(lambda x: json.dumps(x.to_dict()),axis=1)

# 滤掉原来的列
df = df.drop(passengerInfo_cols,axis=1)
df = df.drop(ticketInfo_cols,axis=1)

# 为了能够存入 SQLite，我们将两个字段改为 str 类型
# SQLite 仅支持将数据以 TEXT 形式进行存储
df['passengerInfo'] = df['passengerInfo'].astype(str)
df['ticketInfo'] = df['ticketInfo'].astype(str)

# 将上面的数据写入一张名为 example_table_json 的表
with sqlite3.connect('example.db') as conn:
    df.to_sql('example_table_json', con=conn, if_exists='replace', index=False)

# 链接好到建立好的数据库
connection = sqlite3.connect('example.db')




In [13]:
# 读取泰坦尼克号数据集
df = pd.read_csv('train.csv')
# 将表格中 NaN 值替换为 NULL
df = df.where(pd.notnull(df), None)

# 将数据写入一张名为 example_table 的表
with sqlite3.connect('example.db') as conn:
    df.to_sql('example_table', con=conn, if_exists='replace', index=False)

# 为了下面教程的开展，我们这里需要制作一些新的表格
# 新建一个字段 "CabinType",表示 Cabin 字段的第一个字母
new_df = deepcopy(df)
new_df['CabinType'] = new_df['Cabin'].apply(lambda x: x[0] if x is not None else None)

# 计算每个 CabinType 和 Pclass 分组下的平均船费和生存率
df_gb_pc_ct = new_df.groupby(['Pclass','CabinType'])[['Fare','Survived']].apply(lambda x: np.mean(x)).reset_index()

# 将结果存入数据库,新建一张名为 example_table_gb 的表格
with sqlite3.connect('example.db') as conn:
    df_gb_pc_ct.to_sql('example_table_gb', con=conn, if_exists='replace', index=False)

In [15]:

# 这里我们用 100 个相同查询的查询时间来看反映查询速度的变化

# 查询语句
# 统计每个 pclass 下乘客生存率的方差
query = '''
SELECT t1.Pclass
        ,avg((t2.Survived-t1.Survived)*(t2.Survived-t1.Survived)) AS std -- 方差计算
FROM example_table_gb AS t1
        JOIN example_table AS t2
            ON t1.Pclass = t2.Pclass 
GROUP BY t1.Pclass
'''

# before 记录创建索引前的执行时间
before = []
for i in range(100):
    start_time = time.time()
    _ = connection.execute(query).fetchall()
    before.append(time.time()-start_time)


OperationalError: no such column: t1.Survived

In [ ]:
table_name='example_table_gb'
result=connection.execute(f"PRAGMA table_info({table_name})").fetchall()
result


[(0, 'Pclass', 'INTEGER', 0, None, 0),
 (1, 'CabinType', 'TEXT', 0, None, 0),
 (2, '0', 'REAL', 0, None, 0)]

In [18]:
table_name='example_table'
result=connection.execute(f"PRAGMA table_info({table_name})").fetchall()
result


[(0, 'PassengerId', 'INTEGER', 0, None, 0),
 (1, 'Survived', 'INTEGER', 0, None, 0),
 (2, 'Pclass', 'INTEGER', 0, None, 0),
 (3, 'Name', 'TEXT', 0, None, 0),
 (4, 'Sex', 'TEXT', 0, None, 0),
 (5, 'Age', 'REAL', 0, None, 0),
 (6, 'SibSp', 'INTEGER', 0, None, 0),
 (7, 'Parch', 'INTEGER', 0, None, 0),
 (8, 'Ticket', 'TEXT', 0, None, 0),
 (9, 'Fare', 'REAL', 0, None, 0),
 (10, 'Cabin', 'TEXT', 0, None, 0),
 (11, 'Embarked', 'TEXT', 0, None, 0)]